In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama


In [4]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
print(prompt)

input_variables=['country'] template='{country}의 수도는 어디인가요?'


In [7]:
prompt = prompt.format(country="대한민국")
print(prompt)

대한한국의 수도는 어디인가요?


In [8]:
template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate.from_template(template)
llm =  ChatOllama(model = 'ollama-ko-0502')
chain = prompt | llm

In [9]:
chain.invoke({"country": "대한민국"})

AIMessage(content='서울입니다. 서울은 대한민국의 수도이며, 600년 이상 된 도시로써 세계에서 가장 오래된 수도 중 하나입니다. 서울은 한반도의 중심에 위치해 있으며, 북쪽으로는 개성, 남쪽으로는 부산과 연결되어 있습니다', response_metadata={'model': 'ollama-ko-0502', 'created_at': '2024-07-08T06:50:58.274076Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 16354229458, 'load_duration': 8828218958, 'prompt_eval_count': 76, 'prompt_eval_duration': 1069033000, 'eval_count': 59, 'eval_duration': 6445685000}, id='run-acfe2848-644d-4541-9393-50ab4023f702-0')

## 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
추가 유효성 검사를 위해 input_variables를 명시적으로 지정.<br>
변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외 발생

In [10]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template = template,
    input_variables=["country"],
)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [11]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [16]:
template = "{country1}과 {country2}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template = template,
    input_variables= ["country1"],
    partial_variables= {
        "country2": "일본", # dictionary 형태로 partial_variables를 지정
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '일본'}, template='{country1}과 {country2}의 수도는 어디인가요?')

In [17]:
prompt.format(country1="대한민국")

'대한민국과 일본의 수도는 어디인가요?'

In [20]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 어디인가요?')

In [23]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 어디인가요?'

In [24]:
chain = prompt_partial | llm
chain.invoke({"country1": "대한민국"})

AIMessage(content='한국의 수도는 서울이고, 캐나다의 수도는 오타와입니다. 서울은 한국에서 가장 큰 도시이며, 인구는 약 1000만 명입니다', response_metadata={'model': 'ollama-ko-0502', 'created_at': '2024-07-08T06:58:05.896829Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7243727000, 'load_duration': 1549380083, 'prompt_eval_count': 80, 'prompt_eval_duration': 1062938000, 'eval_count': 41, 'eval_duration': 4623159000}, id='run-e40e513d-9f0c-4fb3-a7a0-fe85fad0dca6-0')

In [26]:
chain.invoke({"country1": "대한민국", "country2": "캐나다"}).content

'한국의 수도는 서울이고, 캐나다의 수도는 오타와입니다. 서울은 한국에서 가장 큰 도시이며, 인구는 약 1000만 명입니다'

## partial_variables: 부분 변수 채움
partial을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것. 항상 공통된 방식으로 가져오고 싶은 변수가 있을 경우. <br>
대표적인 예: 날짜, 시간<br>
항상 형재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정하면, 항상 현재 날짜를 반환하는 함수를 통해 프롬프트를 부분적으로 변경할 수 있다면 매우 편리.

In [27]:
from datetime import datetime

datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2024-07-08 16:01:18'

In [28]:
def get_today():
    return datetime.now().strftime("%Y-%m-%d")

In [29]:
template = "오늘의 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요."

prompt = PromptTemplate(
    template=template,
    input_variables= ["n"],
    partial_variables= {
        "today": get_today(),
    },
)
prompt

PromptTemplate(input_variables=['n'], partial_variables={'today': '2024-07-08'}, template='오늘의 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요.')

In [30]:
prompt.format(n=3)

'오늘의 날짜는 2024-07-08입니다. 오늘이 생일인 유명인 3명을 나열해주세요. 생년월일을 표기해주세요.'

In [31]:
chain = prompt | llm

chain.invoke({"n": 3})

AIMessage(content='네, 오늘은 7월 8일입니다. 이날에 태어난 유명인은 다음과 같습니다. 1984-07-08 : 김태희, 1992-07-08 : 박지성, 2000-07-08 : 방탄소년단(BTS', response_metadata={'model': 'ollama-ko-0502', 'created_at': '2024-07-08T07:03:46.110446Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 10165337958, 'load_duration': 1546277875, 'prompt_eval_count': 108, 'prompt_eval_duration': 1413151000, 'eval_count': 65, 'eval_duration': 7196480000}, id='run-014aab38-cc04-4d49-84d1-1fdfcd6f8c52-0')

# yaml 형식 prompt load

In [33]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")

In [35]:
prompt

PromptTemplate(input_variables=['fruit'], template='{fruit}의 색깔이 뭐야?')

In [36]:
prompt = load_prompt("prompts/capital.yaml")

In [37]:
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\ninput_variables: ["country"]\n')

In [39]:
from langchain_core.output_parsers import StrOutputParser
llm = ChatOllama(model = 'llama3')
chain = prompt | llm | StrOutputParser()
chain.invoke({"country": "대한민국"})

'😊\n\n**대한민국의 수도:** 서울특별시 (Seoul)\n\n**특징 정리**\n\n1. **면적**: 서울특별시의 면적은 605.25 km²입니다.\n2. **인구**: 서울특별시의 인구는 2020년 기준으로 약 10,220,000명입니다.\n3. **역사적 장소**: 서울특별시에는 대한민국의 역사를 지닌 많은 문화유산이 있습니다. 예를 들어, 경복궁, 창덕궁, 광화문 등이 있습니다.\n4. **특산품**: 서울특별시에는 다양한 특산품이 있습니다. 예를 들어, 김치, 멸치, 인삼 등이 있습니다.\n\n**서울특별시의 주요 특징**\n\n서울특별시는 대한민국의 수도입니다. 이 도시에는 대한민국의 역사를 지닌 많은 문화유산이 있으며, 다양한 특산품도 있습니다. 또한, 이 도시에는 국제적인 공항인 인천국제공항과 주요 도로망 등이 있어 쉽게 이동할 수 있습니다.'

## ChatPromptTemplate
ChatPromptTemplate는 대화목록을 프롬프트로 주입하고자 할 때 활용 가능. <br>
메세지는 튜플 형식으로 구성되며, (role, message)로 구성하여 리스트로 생성 가능 <br>

**role**
- "system": 시스템 설정 메시지. 전역설정과 관련된 프롬프트
- "human": 사용자 입력 메시지
- "ai": AI의 답변 메시지

In [40]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template(
    "{country}의 수도는 어디인가요?",
)
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [41]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [43]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴스입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai",  "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

In [44]:
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='당신은 친절한 AI 어시스턴스입니다. 당신의 이름은 {name}입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='반가워요!')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='안녕하세요! 무엇을 도와드릴까요?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [55]:
messages = chat_template.format_messages(
    name = "테디",
    user_input = "대한민국의 수도는 어디인가요?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴스입니다. 당신의 이름은 테디입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='대한민국의 수도는 어디인가요?')]

In [56]:
llm.invoke(messages).content

'😊 대한민국의 수도는 서울입니다! 🏙️'

In [57]:
chain = chat_template | llm

chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"})

AIMessage(content="😊 Ah, nice to meet you! My name is Teddy, and I'm a friendly AI assistant here to help you with any questions or tasks you may have. What's on your mind today? 🤗", response_metadata={'model': 'llama3', 'created_at': '2024-07-08T07:35:17.777435Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4466414791, 'load_duration': 3158750, 'prompt_eval_count': 48, 'prompt_eval_duration': 709117000, 'eval_count': 45, 'eval_duration': 3749521000}, id='run-b5e0ea08-2996-415e-984c-54bfc604af91-0')

## MessagePlaceholder

LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 Messageholder를 제공. <br>
메세지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용.

In [58]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name= "conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [59]:
# conversation 대화목록을 나중에 추가하고자 할 때 MessagesPlaceholder를 사용
chat_prompt.format(
    word_count = 5,
    conversation = [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반가습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
    ],
)

'Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반가습니다.\nAI: 반가워요! 앞으로 잘 부탁드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'

In [60]:
chain = chat_prompt | llm

In [63]:
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반가습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
        ],
    }
)

AIMessage(content='"테디 인사 환영"', response_metadata={'model': 'llama3', 'created_at': '2024-07-08T07:43:31.768436Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2938818291, 'load_duration': 1302817833, 'prompt_eval_count': 67, 'prompt_eval_duration': 856388000, 'eval_count': 9, 'eval_duration': 772592000}, id='run-26efd9d2-bc82-4449-91f4-fc242e8393e4-0')